In [1]:
import os
import torch
import torchvision
import transformers
import sklearn
import scipy
import numpy as np
import pandas as pd
import scanpy as sc
import einops
import matplotlib

ModuleNotFoundError: No module named 'torch'

In [3]:
NAME_DATASET_OUTPUT_PREPROCESS = 'Human_Thymus'

# Input path of the reference/raw data
PATH_PROJECT = "/mnt/DOSI/PLATEFORMES/BIOINFORMATIQUE/04_PROJECT/scLLM"
PATH_EXPERIMENT = os.path.join( PATH_PROJECT, "Human_Thymus_Development_Atlas")
PATH_EXPERIMENT_REFERENCE = os.path.join( PATH_EXPERIMENT, "01_Reference")

PATH_EXPERIMENT_OUTPUT = os.path.join( PATH_EXPERIMENT, "05_Output")
ANALYSIS_NAME = os.path.join( PATH_EXPERIMENT_OUTPUT, "02b_FilterData") 
EXTRA_ANALYSIS_NAME = os.path.join(ANALYSIS_NAME, "Preprocess_Anndata_File_scBERT")

PATH_MODEL = os.path.join( PATH_EXPERIMENT_REFERENCE, "04_Model")
PATH_MODEL_EXTRA = os.path.join(PATH_MODEL, "scBERT-master")
MODEL_PATH_SCBERT = os.path.join( PATH_MODEL, "panglao_pretrain.pth")
PATH_FILE_FINETUNE = os.path.join( PATH_MODEL_EXTRA, "finetune.py")
PATH_FILE_PREDICT = os.path.join( PATH_MODEL_EXTRA, "predict.py")

PATH_OUTPUT_SCBERT = os.path.join( PATH_EXPERIMENT_OUTPUT, "05_scBERT")
os.makedirs(PATH_OUTPUT_SCBERT, exist_ok = True)

# Go to workspace to get the finetuned model
PATH_WORKSPACE_PROJECT = '/mnt/DOSI/PLATEFORMES/BIOINFORMATIQUE/03_WORKSPACE/thyarion/ciml-cb2mproj/Project/scLLM/'
PATH_DATASET_CHOICE = os.path.join(PATH_WORKSPACE_PROJECT, 'Human_Thymus_Development')
PATH_SCRIPT = os.path.join(PATH_DATASET_CHOICE, '03_Script')
PATH_SCRIPT_EXTRA_SCBERT = os.path.join(PATH_SCRIPT,'05_scBERT')
# Il faut changer 06_scBERT qui n'est pas forcément le même nom (en général c'est 05_scBERT)
PATH_MODEL_FINETUNED = os.path.join(PATH_SCRIPT_EXTRA_SCBERT, 'ckpts', 'finetune_best.pth')
PATH_LABEL = os.path.join(PATH_SCRIPT_EXTRA_SCBERT, 'label')
PATH_LABEL_DICT = os.path.join(PATH_SCRIPT_EXTRA_SCBERT, 'label_dict')
PATH_CKPTS = os.path.dirname(PATH_MODEL_FINETUNED)

In [ ]:
# Before start the code, if u encounter a issue (Check if the file "performer_pytorch" the gene2vec are correctly pointed to the repertorie who contain the gene2_vec file)

for z in range (0,3): # Z is using for count the number of replicat about the number of epoch
    for i in range (0, 5) :
            # We set the parameter to get the cross fold validation
            # Select the file that interest us
            VARIABLE_TRAINING_H5AD = os.path.join (EXTRA_ANALYSIS_NAME, NAME_DATASET_OUTPUT_PREPROCESS + '_Training_FOLD_' + str(i) + '_Preprocess.h5ad')
            VARIABLE_TEST_H5AD = os.path.join (EXTRA_ANALYSIS_NAME, NAME_DATASET_OUTPUT_PREPROCESS + '_test_FOLD_' + str(i) + '_Preprocess.h5ad')
        
            # Take the parameter that need for our analysis
            dataset_anndata = sc.read_h5ad(VARIABLE_TRAINING_H5AD)
            VARIABLE_GENE_NUM = dataset_anndata.n_vars
        
            # Repertorie and File output with the prediction (You need to add (+ 1 or more) for the repertorie output)
            if z == 0 :
                VARIABLE_EPOCH = 1
                PATH_OUTPUT_EXTRA = os.path.join(PATH_OUTPUT_SCBERT, 'Run_0'+str(z)+'_epoch_'+str(VARIABLE_EPOCH), 'Result_FOLD_'+str(i)+'_epoch_'+str(VARIABLE_EPOCH))
            elif z == 1 :
                VARIABLE_EPOCH = 5
                PATH_OUTPUT_EXTRA = os.path.join(PATH_OUTPUT_SCBERT, 'Run_0'+str(z+2)+'_epoch_'+str(VARIABLE_EPOCH), 'Result_FOLD_'+str(i)+'_epoch_'+str(VARIABLE_EPOCH))
            elif z == 2 :
                VARIABLE_EPOCH = 10
                PATH_OUTPUT_EXTRA = os.path.join(PATH_OUTPUT_SCBERT, 'Run_0'+str(z+4)+'_epoch_'+str(VARIABLE_EPOCH), 'Result_FOLD_'+str(i)+'_epoch_'+str(VARIABLE_EPOCH))
            
            PATH_OUTPUT_FILE_PREDICTION_SCBERT = os.path.join(PATH_OUTPUT_EXTRA, 'Cell_Type_Predict_FOLD_'+str(i)+'_epoch_'+str(VARIABLE_EPOCH)+'.txt')
            os.makedirs(PATH_OUTPUT_EXTRA, exist_ok = True)

            PATH_OUTPUT_LABELS = PATH_OUTPUT_EXTRA + '/'
            PATH_OUTPUT_LABELS_EXTRA = os.path.join(PATH_OUTPUT_EXTRA, 'ckpts/')
            os.makedirs(PATH_OUTPUT_LABELS_EXTRA, exist_ok = True)
            
            # The command to fine tune scBERT
            !python -m torch.distributed.launch {PATH_FILE_FINETUNE} --model_path {MODEL_PATH_SCBERT} --data_path {VARIABLE_TRAINING_H5AD} --gene_num {VARIABLE_GENE_NUM} --epoch {VARIABLE_EPOCH}

            # We do the prediction
            !python {PATH_FILE_PREDICT} --data_path {VARIABLE_TEST_H5AD} --model_path {PATH_MODEL_FINETUNED} --gene_num {VARIABLE_GENE_NUM} >{PATH_OUTPUT_FILE_PREDICTION_SCBERT}
            
            # We moove the file to the new repertorie
            !cp {PATH_LABEL} {PATH_OUTPUT_LABELS}
            !cp {PATH_LABEL_DICT} {PATH_OUTPUT_LABELS}
            !cp {PATH_MODEL_FINETUNED} {PATH_OUTPUT_LABELS_EXTRA}
            
            # We delete every file in the workspace and put it in the new repertorie
            !rm {PATH_MODEL_FINETUNED} {PATH_LABEL_DICT} {PATH_LABEL}
            !rm -rf {PATH_CKPTS}